In [ ]:
import pandas as pd

import pyteomics.mgf
from tqdm.notebook import tqdm
import seaborn as sns
import matplotlib.pyplot as plt

import pandas_utils as pu
import well_plate

from mapper import combine_polarity

In [ ]:
outfile_no_filter =  r"C:\git\msn_library\data\acquisition_results\202311_all_lib_no_filter.tsv"
outfile_detected = r"C:\git\msn_library\data\acquisition_results\202311_all_lib_only_detected.tsv"
outfile_missing =  r"C:\git\msn_library\data\acquisition_results\202311_all_lib_missing.tsv"


In [ ]:
acquisition_pos = {
  "01_mce_bioactive": r"C:\git\msn_library\library\20231031_mce_library_pos_all_lib_MSn.mgf",
  "03_nih_natural_product": r"C:\git\msn_library\library\20231031_nih_library_pos_all_lib_MSn.mgf",
  "02_mce_scaffold_library": r"C:\git\msn_library\library\20231130_nencka_mce_library_pos_all_lib_MSn.mgf",
  "04_iocb_peptide": r"C:\git\msn_library\library\20231130_iocb_peptide_library_pos_all_lib_MSn.mgf"
}

acquisition_neg = {
  "01_mce_bioactive": r"C:\git\msn_library\library\20231030_mce_library_neg_all_lib_MSn.mgf",
  "03_nih_natural_product": r"C:\git\msn_library\library\20231031_nih_library_neg_all_lib_MSn.mgf",
  "02_mce_scaffold_library": r"C:\git\msn_library\library\20231130_nencka_mce_library_neg_all_lib_MSn.mgf",
  "04_iocb_peptide": r"C:\git\msn_library\library\20231130_iocb_peptide_library_neg_all_lib_MSn.mgf"
}


libraries = {
  "01_mce_bioactive": r"C:\git\msn_library\data\library\mce_library_all_cleaned.tsv",
  "03_nih_natural_product": r"C:\git\msn_library\data\nih\nih_library_new_headers_cleaned_plate7_removed.tsv",
  "02_mce_scaffold_library": r"C:\git\msn_library\data\iocb_libraries\Radim_mce_complete_cleaned.tsv",
  "04_iocb_peptide": r"C:\git\msn_library\data\iocb_libraries\iocb_peptide_library_cleaned.tsv"
}

In [ ]:
def read_mgf(infile) -> pd.DataFrame:
  import re
  rows = []
  counter = 0

  with pyteomics.mgf.MGF(infile) as f_in:
    for spectrum_dict in tqdm(f_in):
      if spectrum_dict is not None:
        rows.append(spectrum_dict["params"])
      else:
        counter += 1

  df = pd.DataFrame(rows)
  if "inchikey" not in df.columns:
    df["inchikey"] = df["inchiaux"]
  if "compound_name" not in df.columns:
    df["compound_name"] = df["name"]
  if "usi" in df.columns:
    df["unique_sample_id"] = ["pluskal{}_id".format(re.search(r'pluskal(.*?)_id', usi).group(1)) for
                              usi in df["usi"]]
  return df

In [ ]:
dfs = []
for key, value in libraries.items():
  df = pu.read_dataframe(value).copy()
  df = df[df["monoisotopic_mass"] > 114]
  df["library"] = key
  df = df.drop_duplicates(["inchikey", "unique_sample_id"])
  dfs.append(df)


metadata_df = pd.concat(dfs, ignore_index=True)


metadata_df

In [ ]:
positive = []
for key, value in acquisition_pos.items():
  df = read_mgf(value)[["inchikey", "unique_sample_id"]].copy()
  df = df.drop_duplicates(["inchikey", "unique_sample_id"])
  positive.append(df)


positive_df = pd.concat(positive, ignore_index=True)


positive_df

In [ ]:
negative = []
for key, value in acquisition_neg.items():
  df = read_mgf(value)[["inchikey", "unique_sample_id"]].copy()
  df = df.drop_duplicates(["inchikey", "unique_sample_id"])
  negative.append(df)


negative_df = pd.concat(negative, ignore_index=True)


negative_df

In [ ]:
mapper = {
  "left_only": "positive",
  "right_only": "negative",
  "both": "both",
}

detect = {
  "left_only": "missing",
  "right_only": "error",
  "both": "detected",
}


detected_df = pd.merge(positive_df, negative_df, indicator="polarity", how="outer", on=["unique_sample_id", "inchikey"])
metadata_detected_df = pd.merge(metadata_df, detected_df, how="left", on=["unique_sample_id", "inchikey"], indicator="detected")
metadata_detected_df["polarity"] = [mapper.get(pol, "missing") for pol in metadata_detected_df["polarity"]]
metadata_detected_df["detected"] = [detect.get(ex, "") for ex in metadata_detected_df["detected"]]
metadata_detected_df

In [ ]:
metadata_detected_df.groupby("detected").count()["inchikey"]

In [ ]:
filtered = metadata_detected_df[metadata_detected_df["detected"] == "detected"]

In [ ]:
missing = metadata_detected_df[metadata_detected_df["detected"] == "missing"]
missing

In [ ]:
pu.save_dataframe(metadata_detected_df, outfile_no_filter)
pu.save_dataframe(filtered, outfile_detected )
pu.save_dataframe(missing, outfile_missing)

## Statistics

In [ ]:
file = r"C:\git\msn_library\data\acquisition_results\all_lib_no_filter.tsv"

In [ ]:
df = pu.read_dataframe(file)

In [ ]:
df

In [ ]:
sub = df.groupby(["library"])

In [ ]:
# unique_dict = {}
# for inchikey, polarity in zip(df["inchikey"], df["polarity"]):
#   oldpolarity = unique_dict.get(inchikey, "missing")
#   unique_dict[inchikey] = combine_polarity(oldpolarity, polarity)
#   
# df["new_polarity"] = [unique_dict.get(inchikey) for inchikey  in df["inchikey"]]
# df = df.sort_values(by=["detected"]).drop_duplicates(["new_polarity", "inchikey"]).sort_index()
# # df[df["inchikey"].duplicated(keep=False)][["inchikey", "polarity", "new_polarity"]]
# df

In [ ]:
def extract_row(libid: str, df: pd.DataFrame) -> dict:
  df = df.drop_duplicates(["inchikey", "unique_sample_id"])
  unique_df = df.drop_duplicates(["inchikey"])
  detected_df = df[df["detected"] == "detected"]
  positive_df = df[(df["polarity"] == "positive") | (df["polarity"] == "both")]
  negative_df = df[(df["polarity"] == "negative") | (df["polarity"] == "both")]
  return {
    "library": libid,
    "total_compounds (unique)": [len(df), len(unique_df)],
    "both_detected_compounds (unique)": [len(detected_df), len(detected_df.drop_duplicates(["inchikey"]))],
    "both_detected_compounds_% (unique)": [len(detected_df.drop_duplicates(["unique_sample_id", "inchikey"])) / len(df) * 100,
                                           len(detected_df.drop_duplicates(["inchikey"]))/ len(unique_df) * 100],
    "pos_detected_compounds (unique)": [len(positive_df.drop_duplicates(["unique_sample_id", "inchikey"])),
                                        len(positive_df.drop_duplicates(["inchikey"]))],
    "pos_detected_compounds_% (unique)": [len(positive_df.drop_duplicates(["unique_sample_id", "inchikey"])) / len(df) * 100,
                                          len(positive_df.drop_duplicates(["inchikey"]))/ len(unique_df) * 100],
    "neg_detected_compounds (unique)": [len(negative_df.drop_duplicates(["unique_sample_id", "inchikey"])),
                                        len(negative_df.drop_duplicates(["inchikey"]))],
    "neg_detected_compounds_% (unique)": [len(negative_df.drop_duplicates(["unique_sample_id", "inchikey"])) / len(df) * 100,
                                          len(negative_df.drop_duplicates(["inchikey"]))/ len(unique_df) * 100],
  }

In [ ]:
lib_rows = []
for group in sub:
  libid = group[0]
  groupdf = group[1]

  row = extract_row(libid, groupdf)
  lib_rows.append(row)

# get summary stats
row = extract_row("Summary", df)
lib_rows.append(row)
lib_rows
statistic_df = pd.DataFrame(lib_rows)

In [ ]:
statistic_df

In [ ]:
df